# Interactively find locations of ground truth and IRL boards

In [1]:
import ipywidgets as widgets
from IPython.display import display
import cv2
import os
import pathlib
import matplotlib.pyplot as plt
import json

In [ ]:
# Global cache for video frames to improve performance
frame_cache = {}

def get_frame(video_path, time_sec):
    """
    Retrieves a specific frame from a video file, using a cache to avoid reloading.
    """
    global frame_cache
    cache_key = (str(video_path), time_sec)
    if cache_key in frame_cache:
        return frame_cache[cache_key]
    
    vidcap = cv2.VideoCapture(str(video_path))
    if not vidcap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return None
        
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    vidcap.set(cv2.CAP_PROP_POS_FRAMES, int(fps * time_sec))
    success, image = vidcap.read()
    vidcap.release()

    if success:
        frame_cache[cache_key] = image
        return image
    else:
        return None

def extract_gt_board(img, gt_board_loc, irl_board_loc):
    """
    Extracts two board regions from an image based on specified coordinates.
    """
    top, bottom, left, right = gt_board_loc
    gt_board = img[top:bottom, left:right, :].copy()
    
    irl_top, irl_bottom, irl_left, irl_right = irl_board_loc
    irl_board = img[irl_top:irl_bottom, irl_left:irl_right, :].copy()

    return gt_board, irl_board

# --- NEW HELPER FUNCTION ---
def create_slider_control(description, slider_params):
    """
    Creates an IntSlider with + and - buttons for precise control.
    
    Returns a tuple: (slider_widget, hbox_ui_element)
    """
    slider = widgets.IntSlider(description=description, **slider_params)
    
    # Define buttons with small layout
    button_layout = widgets.Layout(width='40px')
    down_button = widgets.Button(description='-', layout=button_layout)
    up_button = widgets.Button(description='+', layout=button_layout)

    # Define button click handlers
    def on_down_button_clicked(b):
        slider.value = max(slider.min, slider.value - 1)
        
    def on_up_button_clicked(b):
        slider.value = min(slider.max, slider.value + 1)
        
    down_button.on_click(on_down_button_clicked)
    up_button.on_click(on_up_button_clicked)
    
    # Combine into a horizontal box
    hbox = widgets.HBox([down_button, slider, up_button])
    
    return slider, hbox

def interactive_board_extractor(videos_folder, default_time=3):
    """
    Creates an interactive widget with precise button controls for coordinates.
    """
    video_files = sorted([f for f in os.listdir(videos_folder) if f.endswith('.mp4')])
    if not video_files:
        print(f"No .mp4 videos found in the folder: {videos_folder}")
        return

    video_dropdown = widgets.Dropdown(options=video_files, value=video_files[0], description='Video:')
    time_input = widgets.BoundedFloatText(value=default_time, min=0, max=120, step=5, description='Time (s):')

    # --- Create slider controls using the helper function ---
    gt_y1_slider, gt_y1_ui = create_slider_control('GT y1', {'value': 75, 'min': 0, 'max': 1080})
    gt_y2_slider, gt_y2_ui = create_slider_control('GT y2', {'value': 576, 'min': 0, 'max': 1080})
    gt_x1_slider, gt_x1_ui = create_slider_control('GT x1', {'value': 759, 'min': 0, 'max': 1920})
    gt_x2_slider, gt_x2_ui = create_slider_control('GT x2', {'value': 1258, 'min': 0, 'max': 1920})

    irl_y1_slider, irl_y1_ui = create_slider_control('IRL y1', {'value': 783, 'min': 0, 'max': 1080})
    irl_y2_slider, irl_y2_ui = create_slider_control('IRL y2', {'value': 1080, 'min': 0, 'max': 1080})
    irl_x1_slider, irl_x1_ui = create_slider_control('IRL x1', {'value': 648, 'min': 0, 'max': 1920})
    irl_x2_slider, irl_x2_ui = create_slider_control('IRL x2', {'value': 1271, 'min': 0, 'max': 1920})

    def update(video_name, time_sec, gt_y1, gt_y2, gt_x1, gt_x2, irl_y1, irl_y2, irl_x1, irl_x2):
        video_path = pathlib.Path(videos_folder) / video_name
        image = get_frame(video_path, time_sec)
        if image is None: return
            
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        gt_board_loc = [gt_y1, gt_y2, gt_x1, gt_x2]
        irl_board_loc = [irl_y1, irl_y2, irl_x1, irl_x2]
        gt_board, irl_board = extract_gt_board(image_rgb, gt_board_loc, irl_board_loc)
        
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))
        axs[0].imshow(image_rgb); axs[0].set_title("Full Frame"); axs[0].axis("off")
        axs[1].imshow(gt_board); axs[1].set_title("GT Board"); axs[1].axis("off")
        axs[2].imshow(irl_board); axs[2].set_title("IRL Board"); axs[2].axis("off")
        plt.show()

        video_id = video_name.replace('.mp4', '')
        config = {
            "video_id": video_id, "gt_board_loc": gt_board_loc,
            "irl_board_loc": irl_board_loc, "time_range": [], "step_sec": 1
        }
        print("Copy this configuration:"); print(json.dumps(config, indent=4))

    # Define the UI layout using the generated UI components
    ui = widgets.VBox([
        video_dropdown,
        time_input,
        widgets.HTML(value="<b>Ground Truth (GT) Board Coordinates:</b>"),
        widgets.HBox([gt_y1_ui, gt_y2_ui]),
        widgets.HBox([gt_x1_ui, gt_x2_ui]),
        widgets.HTML(value="<b>In-Real-Life (IRL) Board Coordinates:</b>"),
        widgets.HBox([irl_y1_ui, irl_y2_ui]),
        widgets.HBox([irl_x1_ui, irl_x2_ui])
    ])
    
    # Link the UI to the update function, connecting to the sliders themselves
    out = widgets.interactive_output(update, {
        'video_name': video_dropdown, 'time_sec': time_input,
        'gt_y1': gt_y1_slider, 'gt_y2': gt_y2_slider, 'gt_x1': gt_x1_slider, 'gt_x2': gt_x2_slider,
        'irl_y1': irl_y1_slider, 'irl_y2': irl_y2_slider, 'irl_x1': irl_x1_slider, 'irl_x2': irl_x2_slider
    })
    
    display(ui, out)

# --- Execution ---
current_dir = pathlib.Path.cwd()
root_dir = current_dir.parent 
videos_dir = root_dir / "data" / "videos"

if videos_dir.exists():
    interactive_board_extractor(videos_dir)
else:
    print(f"Error: The specified videos directory does not exist: {videos_dir}")

Output()

In [ ]:

# Global cache for video frames to improve performance
frame_cache = {}

def get_frame(video_path, time_sec):
    """
    Retrieves a specific frame from a video file, using a cache to avoid reloading.
    """
    global frame_cache
    cache_key = (str(video_path), time_sec)

    # Check if the frame is already in the cache
    if cache_key in frame_cache:
        return frame_cache[cache_key]

    # If not in cache, load from video file
    vidcap = cv2.VideoCapture(str(video_path))
    if not vidcap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return None
        
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    vidcap.set(cv2.CAP_PROP_POS_FRAMES, int(fps * time_sec))
    success, image = vidcap.read()
    vidcap.release()

    if success:
        # Store the newly loaded frame in the cache
        frame_cache[cache_key] = image
        return image
    else:
        return None

def extract_gt_board(img, gt_board_loc, irl_board_loc):
    """
    Extracts two board regions from an image based on specified coordinates.
    """
    top, bottom, left, right = gt_board_loc
    gt_board = img[top:bottom, left:right, :].copy()
    
    irl_top, irl_bottom, irl_left, irl_right = irl_board_loc
    irl_board = img[irl_top:irl_bottom, irl_left:irl_right, :].copy()

    return gt_board, irl_board

def interactive_board_extractor(videos_folder, default_time=3):
    """
    Creates an interactive widget to select a video, a frame, and define
    bounding boxes for two board regions, displaying the result and the configuration.
    """
    # List all mp4 files in the folder
    video_files = sorted([f for f in os.listdir(videos_folder) if f.endswith('.mp4')])
    if not video_files:
        print(f"No .mp4 videos found in the folder: {videos_folder}")
        return

    video_dropdown = widgets.Dropdown(
        options=video_files,
        value=video_files[0],
        description='Video:',
        disabled=False,
    )
    time_slider = widgets.FloatSlider(value=default_time, min=0, max=60, step=0.1, description='Time (s):', continuous_update=False)

    # Sliders for GT bounding box coordinates
    gt_y1 = widgets.IntSlider(value=29, min=0, max=500, step=2, description='GT y1')
    gt_y2 = widgets.IntSlider(value=186, min=0, max=500, step=2, description='GT y2')
    gt_x1 = widgets.IntSlider(value=254, min=0, max=800, step=2, description='GT x1')
    gt_x2 = widgets.IntSlider(value=411, min=0, max=800, step=2, description='GT x2')

    # Sliders for IRL bounding box coordinates
    irl_y1 = widgets.IntSlider(value=200, min=0, max=500, step=2, description='IRL y1')
    irl_y2 = widgets.IntSlider(value=300, min=0, max=500, step=2, description='IRL y2')
    irl_x1 = widgets.IntSlider(value=200, min=0, max=800, step=2, description='IRL x1')
    irl_x2 = widgets.IntSlider(value=450, min=0, max=800, step=2, description='IRL x2')

    def update(video_name, time_sec, gt_y1, gt_y2, gt_x1, gt_x2, irl_y1, irl_y2, irl_x1, irl_x2):
        video_path = pathlib.Path(videos_folder) / video_name
        image = get_frame(video_path, time_sec)
        
        if image is None:
            print(f"Could not load frame from {video_name} at {time_sec}s.")
            return
            
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        gt_board_loc = [gt_y1, gt_y2, gt_x1, gt_x2]
        irl_board_loc = [irl_y1, irl_y2, irl_x1, irl_x2]
        
        gt_board, irl_board = extract_gt_board(image_rgb, gt_board_loc, irl_board_loc)
        
        # Plotting the images
        fig, axs = plt.subplots(1, 3, figsize=(15, 5))
        axs[0].imshow(image_rgb)
        axs[0].set_title("Full Frame")
        axs[1].imshow(gt_board)
        axs[1].set_title("GT Board")
        axs[2].imshow(irl_board)
        axs[2].set_title("IRL Board")
        for ax in axs:
            ax.axis("off")
        plt.show()

        # --- Create and print the configuration dictionary ---
        video_id = video_name.replace('.mp4', '')
        
        config = {
            "video_id": video_id,
            "gt_board_loc": gt_board_loc,
            "irl_board_loc": irl_board_loc,
            "time_range": [],
            "step_sec": 1
        }
        
        # Print the configuration as a formatted string for easy copying
        print("Copy this configuration:")
        print(json.dumps(config, indent=4))


    # Define the UI layout
    ui = widgets.VBox([
        video_dropdown,
        time_slider,
        widgets.HTML(value="<b>Ground Truth (GT) Board Coordinates:</b>"),
        widgets.HBox([gt_y1, gt_y2, gt_x1, gt_x2]),
        widgets.HTML(value="<b>In-Real-Life (IRL) Board Coordinates:</b>"),
        widgets.HBox([irl_y1, irl_y2, irl_x1, irl_x2])
    ])
    
    # Link the UI to the update function
    out = widgets.interactive_output(update, {
        'video_name': video_dropdown,
        'time_sec': time_slider,
        'gt_y1': gt_y1, 'gt_y2': gt_y2, 'gt_x1': gt_x1, 'gt_x2': gt_x2,
        'irl_y1': irl_y1, 'irl_y2': irl_y2, 'irl_x1': irl_x1, 'irl_x2': irl_x2
    })
    
    # Display the UI and the output
    display(ui, out)

# --- Execution ---
# NOTE: You may need to adjust this path based on your directory structure
current_dir = pathlib.Path.cwd()
# Assuming your "data" folder is in the parent directory of your notebook's location
root_dir = current_dir.parent 
videos_dir = root_dir / "data" / "videos"

# Check if the directory exists before running
if videos_dir.exists():
    interactive_board_extractor(videos_dir)
else:
    print(f"Error: The specified videos directory does not exist: {videos_dir}")
    print("Please check the path.")

In [ ]:
def test_extract_gt_board():
    current_dir = pathlib.Path.cwd()
    base_dir = current_dir.parent
    video_name = "rrPfmSWlAPM"
    video_path = base_dir / "data" / "videos" / f"{video_name}.mp4"

    video_min = 0
    video_seconds = 3
    time_sec = video_min * 60 + video_seconds
    image = get_frame(video_path, time_sec)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    plt.imshow(image)
    plt.axis("off")
    plt.show()

    # Extract ground truth and IRL board
    gt_board_loc = [29, 186, 254, 411]  # [y1, y2, x1, x2]
    irl_board_loc = [200, 300, 200, 450]  # [y1, y2, x1, x2]

    gt_board, irl_board = extract_gt_board(image, gt_board_loc, irl_board_loc)

    plt.imshow(gt_board)
    plt.axis("off")
    plt.show()

    plt.imshow(irl_board)
    plt.axis("off")
    plt.show()